## Practice: Python list operations

In [10]:
# Start with a sample list
L = [3, 1, 4, 1, 5]
print("Initial L =", L)

# append(x): add one item to the end
L.append(9)
print("after append(9):", L)

# count(x): number of occurrences of x
print("count(1):", L.count(1))

# extend(iterable): append all elements of iterable
L.extend([4, 6])
print("after extend([2,6]):", L)

# index(x): first index of value x (raises ValueError if not found)
print("index(4):", L.index(4),": first match index")

# insert(i, x): insert x before position i
L.insert(2, 7)
print("after insert(2,7):", L)

# pop(): remove and return last item (pop(i) removes at index i)
last = L.pop()
print("pop() returned:", last, "| L =", L)

first = L.pop(0)
print("pop(0) returned:", first, "| L =", L)

# remove(x): remove first occurrence of x (raises ValueError if not found)
L.remove(1)
print("after remove(1):", L,": remove an 1, error reported if element doesn't exist")

# copy(): shallow copy of the list
C = L.copy()
print("copy C =", C, "| (C is a different object from L?)", C is not L)

# reverse(): reverse list in place
C.reverse()
print("after C.reverse():", C)

# sort(): sort list in place (default ascending)
C.sort()
print("after C.sort():", C)

# clear(): remove all items
C.clear()
print("after C.clear():", C)


Initial L = [3, 1, 4, 1, 5]
after append(9): [3, 1, 4, 1, 5, 9]
count(1): 2
after extend([2,6]): [3, 1, 4, 1, 5, 9, 4, 6]
index(4): 2 : first match index
after insert(2,7): [3, 1, 7, 4, 1, 5, 9, 4, 6]
pop() returned: 6 | L = [3, 1, 7, 4, 1, 5, 9, 4]
pop(0) returned: 3 | L = [1, 7, 4, 1, 5, 9, 4]
after remove(1): [7, 4, 1, 5, 9, 4] : remove an 1, error reported if element doesn't exist
copy C = [7, 4, 1, 5, 9, 4] | (C is a different object from L?) True
after C.reverse(): [4, 9, 5, 1, 4, 7]
after C.sort(): [1, 4, 4, 5, 7, 9]
after C.clear(): []


## List of length "N" Practice

Takes 7 repeated trials and measure their time. Median time is printed and for comparison.

Cost of removing elements: pop() vs pop(0):

In [11]:
# Benchmarks for Lecture 03 lab (all three bullets)
# N = 16, 32, 64, ... (doubling). Run this as ONE cell in your .ipynb.

import random
import time
import statistics

# -----------------------
# Settings (edit if needed)
# -----------------------
N0 = 16
num_levels = 10          # Ns = 16 * 2^k, k=0..num_levels-1
repeats = 7              # take median over repeats (odd is nice)
inner_copy = 20000       # repeat copy/reverse many times per timing to reduce noise
seed = 0

# pop(0) can blow up (O(N^2) total to empty list). Cap its max N to avoid long runs.
max_N_for_pop0 = 4096

# -----------------------
# Helpers
# -----------------------
def make_random_list(n, seed=0):
    rng = random.Random(seed)
    return [rng.randrange(0, 10**9) for _ in range(n)]

def time_median(fn, repeats):
    ts = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        fn()
        ts.append(time.perf_counter() - t0)
    return statistics.median(ts)

def Ns_geometric(N0, num_levels):
    return [N0 * (2**k) for k in range(num_levels)]

Ns = Ns_geometric(N0, num_levels)

# -----------------------
# 1) Cost of removing elements: pop() vs pop(0)
# -----------------------

print("1) Removing all elements")
print(f"{'N':>6}  {'pop() median time (s)':>16}  {'pop(0) median time (s)':>18}  {'ratio pop0/pop':>16}")
print("-" * 62)

for n in Ns:
    def run_pop():
        L = make_random_list(n, seed)
        while L:
            L.pop()

    def run_pop0():
        L = make_random_list(n, seed)
        while L:
            L.pop(0)

    t_pop = time_median(run_pop, repeats)
    t_pop0 = time_median(run_pop0, repeats)
    ratio = (t_pop0 / t_pop) if t_pop > 0 else float("inf")
    print(f"{n:6d}  {t_pop:16.6e}  {t_pop0:18.6e}  {ratio:16.2f}")

print("\nNote: pop(0) takes longer time because of shifting.\n")


1) Removing all elements
     N  pop() median time (s)  pop(0) median time (s)    ratio pop0/pop
--------------------------------------------------------------
    16      1.140000e-05        1.150000e-05              1.01
    32      1.540000e-05        1.630000e-05              1.06
    64      2.430000e-05        2.520000e-05              1.04
   128      4.260000e-05        4.230000e-05              0.99
   256      7.820000e-05        8.250000e-05              1.05
   512      1.510000e-04        1.569000e-04              1.04
  1024      2.815000e-04        3.178000e-04              1.13
  2048      5.600000e-04        3.686500e-03              6.58
  4096      8.794000e-04        1.481480e-02             16.85
  8192      1.729000e-03        6.152270e-02             35.58

Note: pop(0) takes longer time because of shifting.



Slicing vs explicit copy: A=L[:] vs B=list(L)

In [12]:
# -----------------------
# 2) Slicing vs explicit copy: A=L[:] vs B=list(L)
# -----------------------

print("2) Copying a list (in microseconds, taking median)")
print(f"{'N':>6}  {'L[:] (µs)':>18}  {'list(L) (µs)':>20}  {'ratio list/slice':>18}")
print("-" * 70)

for n in Ns:
    L = make_random_list(n, seed)

    def run_slice():
        A = L[:]      # slicing copy

    def run_list():
        B = list(L)   # constructor copy

    t_slice = time_median(run_slice, repeats)
    t_list  = time_median(run_list,  repeats)
    print(f"{n:6d}  {t_slice*1e6:18.2f}  {t_list*1e6:20.2f}  {t_list/t_slice:18.3f}")

print("\nNote: both are O(N) copies; slicing is often slightly faster but not guaranteed.\n")


2) Copying a list (in microseconds, taking median)
     N           L[:] (µs)          list(L) (µs)    ratio list/slice
----------------------------------------------------------------------
    16                0.30                  0.20               0.667
    32                0.10                  0.20               2.000
    64                0.20                  0.20               1.000
   128                0.40                  0.30               0.750
   256                0.70                  0.50               0.714
   512                1.00                  1.00               1.000
  1024                2.10                  2.00               0.952
  2048                4.10                  4.00               0.976
  4096                7.20                  7.30               1.014
  8192               14.70                 15.20               1.034

Note: both are O(N) copies; slicing is often slightly faster but not guaranteed.



In-place vs out-of-place reverse: L.reverse() vs R=L[::-1]

In [13]:
# -----------------------
# 3) In-place vs out-of-place reverse: L.reverse() vs R=L[::-1]
# -----------------------
print("3) Reversing a list (in microseconds, taking median)")
print(f"{'N':>6}  {'reverse() (µs)':>20}  {'L[::-1] (µs)':>20}  {'ratio out/inplace':>20}")
print("-" * 76)

for n in Ns:
    L0 = make_random_list(n, seed)

    def run_inplace():
        L = list(L0)
        L.reverse()        # in-place

    def run_outplace():
        L = list(L0)
        R = L[::-1]        # out-of-place (new list)

    t_in = time_median(run_inplace, repeats)
    t_out = time_median(run_outplace, repeats) 
    print(f"{n:6d}  {t_in*1e6:20.2f}  {t_out*1e6:20.2f}  {t_out/t_in:20.3f}")

print("\nNote: both are O(N). reverse() is in-place (O(1) extra memory), "
      "while L[::-1] allocates a new list (O(N) extra memory).")


3) Reversing a list (in microseconds, taking median)
     N        reverse() (µs)          L[::-1] (µs)     ratio out/inplace
----------------------------------------------------------------------------
    16                  0.30                  0.20                 0.667
    32                  0.20                  0.30                 1.500
    64                  0.30                  0.30                 1.000
   128                  0.40                  0.60                 1.500
   256                  0.50                  0.90                 1.800
   512                  1.00                  1.40                 1.400
  1024                  1.80                  3.10                 1.722
  2048                  4.10                  6.40                 1.561
  4096                  8.30                 14.00                 1.687
  8192                 15.30                 26.60                 1.739

Note: both are O(N). reverse() is in-place (O(1) extra memory), wh